In [ ]:
import csv
import os
import speech_recognition as sr
import gradio as gr

# Load inventory from CSV
def load_inventory(file_path):
    inventory = {}
    try:
        with open(file_path, mode='r') as file:
            reader = csv.DictReader(file)
            for row in reader:
                product = row["product"]
                inventory[product] = {"quantity": int(row["quantity"]), "price": float(row["price"])}
    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except Exception as e:
        print(f"Error loading inventory: {str(e)}")
    return inventory

# Update the file path to the resources folder
csv_file_path = os.path.join('Resources', 'mock_inventory.csv')
inventory = load_inventory(csv_file_path)

def process_command(command):
    tokens = command.lower().split()
    if "find" in tokens and "product" in tokens:
        return "Sure, I can help you find a product. What are you looking for?"
    else:
        return "I'm sorry, I didn't understand that. Can you please repeat?"

def speech_to_text(audio):
    recognizer = sr.Recognizer()
    try:
        # Convert the audio to a format that speech_recognition can use
        with sr.AudioFile(audio) as source:
            audio_data = recognizer.record(source)
        # Recognize the speech
        text = recognizer.recognize_google(audio_data)
        return text
    except sr.UnknownValueError:
        return "Sorry, I could not understand the audio."
    except sr.RequestError as e:
        return f"Could not request results; {e}"

def voice_interface(audio):
    if audio is not None:
        text = speech_to_text(audio)
        response = process_command(text)
        return f"You said: {text}\
Response: {response}"
    else:
        return "No audio input received."

# Launch the Gradio interface
# The 'source' argument has been removed from gr.Audio
interface = gr.Interface(
    fn=voice_interface,
    inputs=gr.Audio(type="filepath"),  
    outputs="text",
    title="Ask Paxil"
)

interface.launch(share=True)

print("Gradio interface launched. You can now interact with the voice assistant.")